In [8]:
import numpy as np
from pyproj import CRS, Transformer, Proj

In [9]:
# Grid files: https://github.com/OSGeo/proj-datumgrid
# NGS geoid models: https://geodesy.noaa.gov/GEOID/GEOID18/

# Hungarian model: http://www.agt.bme.hu/osgeolab/pub/takacs_siki_transzformacio_2017.pdf
# EHT and VITEL's geoid model has centimeter level issues: file:///home/zoltan/Downloads/proj_poszter_3d.pdf

nadgrids = '/home/zoltan/Repo/uni/data/etrs2eov_notowgs.gsb'# XY grid
geoidgrids = '/home/zoltan/Repo/uni/data/geoid_eht2014.gtx' # geoid model for Z transformation
crs = Proj(init='EPSG:23700', nadgrids=nadgrids) # HD72 / EOV
crs_geoid = Proj(init='EPSG:23700', nadgrids=nadgrids, geoidgrids=geoidgrids) # HD72 / EOV

/home/zoltan/env/uni/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/zoltan/env/uni/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [10]:
to_geoid = Transformer.from_proj(crs, crs_geoid)

In [11]:
easting = 650684.000
northing = 237444.000 
height = 137.177 
p_geoid = to_geoid.transform(easting, northing, height)
geoid_shift = height - p_geoid[2]
print(geoid_shift)

43.63338949961758


In [12]:
site_bound = 500 # 1000 * 1000 m2 = 100 hectare ~= 250 acres
y_grid = np.arange(easting - site_bound, easting + site_bound, 5.0)
x_grid = np.arange(northing - site_bound, northing + site_bound, 5.0)
z_grid = np.ones((x_grid.shape[0], 1)) * height
coors_geoid = to_geoid.transform(y_grid, x_grid, z_grid)
dh_max = np.max(np.abs(coors_geoid[2] - z_grid + geoid_shift))
print('dh_max: ', dh_max)

dh_max:  0.008177140061647492


In [13]:
# GEOID18 model accuracy: source: https://geodesy.noaa.gov/GEOID/GEOID18/geoid18_tech_details.shtml
# The relative accuracy of GEOID18 to NAVD 88 is characterized by a estimated 
# uncertainty of +/-1.27 centimeters nationwide.  These errors range from 
# 1.9 mm to 6.25 cm and are inconsistently distributed throughout the grids.

In [39]:
crs = Proj(init='EPSG:5972') # HD72 / EOV
crs_geoid = Proj(init='EPSG:4258') 
to_geoid = Transformer.from_proj(crs, crs_geoid)

/home/zoltan/env/uni/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/zoltan/env/uni/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [40]:
easting = 716552.654289
northing = 6954136.681566
height = 10 
p_geoid = to_geoid.transform(easting, northing, height)
print(p_geoid)

(13.2275390625018, 62.65396335371094, 10.0)


In [45]:
from PIL import Image
im = Image.open('/home/zoltan/Downloads/no_kv_HREF2018A_NN2000_EUREF89.tif')
print(im.info)

{'compression': 'tiff_adobe_deflate', 'dpi': (1, 1), 'resolution': (1, 1)}
